# Datová analýza exoplanetárního datasetu</h1>
## Semestrální práce z předmětu 4ST101
### *Autor: Jakub Vítovec*

## 1. Charakteristika zpracovávaných dat.
Ve své semestrální práci se budu zabývat daty získanými z veřejné databáze [Keggle](https://www.kaggle.com/). Konkrétní dataset je dostupný na URL [https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system](https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system) a byl vytvořen uživatelem Sathyanarayan Rao z dat publikovaných vesmírnou agenturou NASA. Celkem se v datasetu nachází 32552 různých dosud objevených exoplanet a základní data o nich.
Exoplaneta je planeta, která obíhá okolo jiné hvězdy, než okolo Slunce. První exoplaneta byla objevena v roce 1998 a počet jejich počet stále roste.